In [ ]:
# %load scrape_random.py
# file_max = 3000
# counter = 0
# import requests
# import time

# random_url = "http://en.wikipedia.org/wiki/Special:Random"

# while counter < file_max:
#     if counter % 1000 == 0:
#         print("On {}".format(counter))
#     try:
#         data = requests.get(random_url)
#         content = data.content.decode("utf-8")
#         if len(content) > 0:
#             end = data.url.split("/")[-1]
#             with open("wiki/{}.html".format(end), "w+") as f:
#                 f.write(content)
#                 counter += 1
#     except Exception as e:
#         print(e)
#     time.sleep(1.5)

In [2]:
import os
from bs4 import BeautifulSoup
import concurrent.futures
import time


In [3]:
files = [i for i in os.listdir("wiki")]
print(len(files))

with open(os.getcwd() +"/wiki/"+ files[0],"r") as f:
    print(f.read())
    

1144
<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>United Nations Security Council Resolution 1049 - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"United_Nations_Security_Council_Resolution_1049","wgTitle":"United Nations Security Council Resolution 1049","wgCurRevisionId":825726281,"wgRevisionId":825726281,"wgArticleId":27023580,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from April 2012","1996 United Nations Security Council resolutions","1996 in Burundi","United Nations Security Council resolutions concerning Burundi","March 1996 events"],"wgBreakFrames":false,"wgPageCont

In [5]:
content = list()
for file in files:
    with open(os.getcwd()+"/wiki/"+file,"r") as f:
        content.append(f.read())
articles = [i.replace("wiki/","").replace(".html","") for i in files]
print(articles[:15])

['United_Nations_Security_Council_Resolution_1049', 'Furubira_District,_Hokkaido', 'Bill_Brewster_(politician)', 'Valentin_Yanin', 'Kings_XI_Punjab_in_2014', 'William_Harvey_Lillard', 'Radial_Road_3', 'George_Weldrick', 'WACX', 'Benjamin_Plim_Bellamy', 'Zgornji_Otok', 'Blue_Heelers_(season_8)', 'Taggen_Nunatak', '1951_National_League_tie-breaker_series', 'List_of_number-one_singles_of_1993_(Finland)']


In ***articles*** in some cases a lot of url encoded characters. We should remove them or better- replace them.

In [12]:
def html_parse(html_file):
    soup = BeautifulSoup(html_file, 'html.parser')
    extracted = str(soup.find_all("div", id="content")[0])
    return extracted

start = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
parsed = list(pool.map(html_parse,content))
print(time.time() - start)    

41.541186571121216


5 workers ~ 41 seconds
10 workers ~ 46 seconds
2 workers ~ 41 seconds
1 worker ~ 57 seconds

It seems that we are using 2 core processor because more than 2 workers only increase runtime.

In [13]:
def count_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    tags = {}
    for tag in soup.find_all():
        if tag.name not in tags:
            tags[tag.name] = 0
        tags[tag.name] += 1
    return tags

start = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
tags = pool.map(count_tags, parsed)
tags = list(tags)

tag_counts = {}
for tag in tags:
    for k,v in tag.items():
        if k not in tag_counts:
            tag_counts[k] = 0
        tag_counts[k] += v
end = time.time()

print(end - start)
tag_counts

21.297205686569214


{'a': 186763,
 'abbr': 4184,
 'annotation': 3,
 'area': 43,
 'audio': 3,
 'b': 16207,
 'bdi': 4,
 'big': 78,
 'blockquote': 64,
 'br': 5732,
 'caption': 236,
 'center': 162,
 'cite': 4139,
 'code': 123,
 'dd': 1605,
 'del': 2,
 'div': 33168,
 'dl': 531,
 'dt': 359,
 'font': 40,
 'h1': 1144,
 'h2': 4648,
 'h3': 935,
 'h4': 131,
 'h5': 4,
 'h6': 1,
 'hr': 60,
 'i': 21581,
 'img': 7467,
 'li': 100967,
 'map': 4,
 'math': 3,
 'mfrac': 1,
 'mi': 7,
 'mn': 2,
 'mo': 5,
 'mrow': 11,
 'mstyle': 3,
 'msub': 3,
 'munderover': 1,
 'noscript': 1144,
 'ol': 977,
 'p': 8972,
 'pre': 1,
 'q': 82,
 'rb': 16,
 'rp': 32,
 'rt': 16,
 'ruby': 16,
 's': 12,
 'samp': 2,
 'semantics': 3,
 'small': 3635,
 'source': 4,
 'span': 77083,
 'strong': 602,
 'sub': 209,
 'sup': 12695,
 'table': 4590,
 'tbody': 468,
 'td': 62798,
 'th': 16808,
 'tr': 30565,
 'u': 53,
 'ul': 12883,
 'wbr': 101}

In [20]:
from collections import Counter
import re

def count_words(html):
    soup = BeautifulSoup(html, 'html.parser')
    data = soup.get_text().lower()
    data = re.sub("\W+"," ", data)
    data = data.split(" ")
    data = [i for i in data if len(i) > 3]    
    return Counter(data)

start = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
words = list(pool.map(count_words, parsed))


words_counts = {}
for word in words:
    for k,v in word.items():
        if k not in words_counts:
            words_counts[k] = 0
        words_counts[k] += v
end = time.time()

print(end - start)
words_counts

23.540266036987305


{'sensors': 7,
 'beziehung': 1,
 'delegation': 8,
 'deeti': 1,
 'textthailand': 1,
 'stubs1972': 1,
 'subsription': 1,
 'klukowo': 6,
 'musicale': 1,
 'vagana': 3,
 'bysamuel': 17,
 'hiros': 1,
 'saybagh': 1,
 'munichhidden': 1,
 'tufan': 5,
 'juruá': 1,
 'blackhall': 1,
 'somatrem': 1,
 'dhayalan': 16,
 'gerstheim': 2,
 'sola': 2,
 'stubstongan': 1,
 'gretzky': 1,
 'indianola': 1,
 'britainparliament': 1,
 'alumnicornell': 1,
 'baroghil': 1,
 'gavorrano': 1,
 'pyrochta': 6,
 'reyhan': 1,
 '13069': 1,
 'politiciansnew': 1,
 'hossain': 4,
 'kukadoli': 1,
 'baqerabad': 2,
 'lustre': 1,
 'bromwich': 8,
 'rewriteall': 1,
 'managed': 35,
 'phrases': 9,
 '09917': 2,
 'gringos': 1,
 'campo': 5,
 '1087': 4,
 'ochen': 1,
 'hartigan': 1,
 'nationalization': 1,
 'arrosès': 1,
 'discs': 4,
 'mareshk': 1,
 'rosedale': 2,
 'subscription': 35,
 'shekhar': 6,
 'negatives': 2,
 'gowanus': 1,
 'pcmag': 1,
 'windows': 39,
 'sebree': 1,
 '7cbad3d9': 1,
 'sevilla': 1,
 'nfpa': 1,
 'janmorris': 1,
 'rasga':

with 3 letters ~ 23.5 sec runtime  
with 5 letters ~ 23 sec runtime  
with 7 letters ~ 22.8 sec runtime  

Changing to 7 letters differs runtime by 0.2 miliseconds.  
While 3 to 6 change does - 0.5 milisec runtime.  

Basicly I can explain that **the more letters** allow by filter **the less words** you going to find, the less program would summarize that dict with word counts.   

We only win time at summarize data of all dicts. 